# 语言模型实现文本生成
https://blog.csdn.net/Yellow_python/article/details/89088854

In [1]:
from collections import Counter
from jieba import lcut
from random import choice

## 语料

In [2]:
corpus = '''
风儿吹过，纷扬我的思念，小雨洒落，浸润我的牵挂，雷声哄响，共鸣所有的记忆，想你了，我的朋友，无论你身处何方，都祝愿你天天开心，幸福无限，常联系！
时光的记忆曲线，将美好串联；人生的情感轨迹，将幸福标注；生活的点点滴滴，将快乐洋溢；生命的友情之花，将情谊芬芳。朋友，惟愿你天天开心，永享幸福！
人生有时有无奈，爱不想爱的人，走不想走的路，做不想做的事，操不想操的心。还好有你这个朋友陪在身边，让无奈的人生平添了许多的快乐和精彩。愿你一切安好！
微笑是个搬运工，可以随时搬走压力；微笑是个维修工，可以轻松修复哀痛；微笑是个清洁工，可以认真扫除烦忧。朋友，请每天保持微笑，成为快乐的保管工，幸福的加油工！
上班太烦，工作太乱，领导太粘，生活太酸，怎么办？别着急，周末到了，开心为你解烦，快乐为你解难，开心让你缠绵，健康与你作伴，你就乐呵乐呵着吧！
日头不眨眼，火热在人间。褪去厚衣衫，林荫寻悠闲。清心又寡欲，坐看风云变。烦扰多少事，轻轻付笑谈。以禅养心术，处之皆安然。盛夏时分，愿君惬意！
微微的清风吹散烦忧，暖暖的阳光蒸发惆怅；真真的祝福轻轻送达，诚诚的情谊温暖心田。朋友，愿你保重身体，健康平安，幸福快乐，顺心如意，心想事成！
时间，忘却不了美好的回忆；岁月，冲淡不了友情的芬芳；距离，阻断不了心灵的感应；忙碌，断绝不了彼此的情谊。朋友，惟愿你时时开怀，幸福快乐每一天！
阳光洒在心中，惊醒一帘幽梦；问候如沐清风，蓝了朗朗晴空；泛起甜甜笑容，心事一切随风。愿你开启快乐一天！
岁月匆匆人渐老，日子天天新模样，愿你容颜天天焕发，永远美丽又俊俏，祝福你美丽的姑娘，青春永驻，白马王子天天围着你，爱情甜甜蜜蜜，生活幸福美满！
有一种默契，叫做我挨骂你顶着；有一种和谐，叫做我坐着你站着；有一种情谊，叫做我吃肉你喝汤；有一种祝福，叫做我发短信要你开开心心！
一周时间不算长，心里想你直痒痒，联络短信要常发，寄托情感很重要，人生短短几十载，朋友祝福像甘泉，祝你天天喜洋洋，健康平安常相伴！
天冷了，唐僧添了一件小棉袍，悟空添了一条小棉裤，沙僧添了一个小棉帽，八戒，你那双小手，别只顾着玩手机看短信呀，记得戴上一幅小手套。
丝丝情感，融汇真挚的元素；滴滴友情，酝酿情谊的芬芳；点点思绪，汇成牵挂的浪花；声声问候，化作无限的美好。朋友，愿你开怀，时刻如意！
伴着清晨的阳光，伴着愉快的心情，新的一天又开始了。在这新的一天里，亲爱的朋友我又在想你，编个短信发给你，祝你健康好身体，快乐又美丽！
秋风从南起，落叶一夜袭，风光带凉意，秋意有几许，花落又一季，韶华君应惜，朋友应相忆，殷殷祝福语，纷飞了心绪，问候暖君心，盼尔总如意！
天热了，我托清风，为你殷勤打扇；闲暇时，我用问候，与你绵绵攀谈；烦躁了，我洒细雨，为你洗去尘世如烟。时值酷暑，愿你心情惬意，更胜从前！
一壶老酒，散发友情的醇香；一首老歌，唱出曾经的美好；一份思念，道出朋友的牵挂；一声问候，则满载朋友的情谊。朋友，愿你安好，没事常联系！
微微的清风吹散烦忧，暖暖的阳光蒸发惆怅；真真的祝福轻轻送达，诚诚的情谊温暖心田。朋友，愿你保重身体，健康平安，幸福快乐，顺心如意，心想事成！
长江冲不尽思念，大海涛不尽离愁，无论天涯海角荒滩戈壁，我的心就象空中的云，云中的雨丝丝缕缕缠缠绵绵，萦绕着沐浴着你。祝福你，我的朋友幸福平安！
风儿吹过，纷扬我的思念，小雨洒落，浸润我的牵挂，雷声哄响，共鸣所有的记忆，想你了，我的朋友，无论你身处何方，都祝愿你天天开心，幸福无限，常联系！
天上的星星很多，可是多不过我的思念，雨中的雷声很大，可是大不过我的牵挂，爱里的誓言最真，可是真不过我的祝福，朋友，愿你天天都有好心情，事事都如意！
我对你的思念象深谷里的幽兰，淡淡的香气笼罩着你，而祝福是无边的关注一直飘到你心底。愿我的爱陪伴你走过新的一年！
在这美丽温馨的世界里，聆听大地的祝福，这样洋溢希望的一年，祝愿你新年快乐，生活幸福！
新年将至，献上一份祝福。期盼：每一天，幸福之神都陪伴着你；每一刻，快乐之神都保佑着你。兔年新年快乐！
多一份欢欣就多一份快乐就多一份如意。愿节日的欢乐，新年的快乐，永远伴随着你。
漫天的雪花，是飞舞的音符，以思念谱成乐章，用祝福奏出期盼，带给你，欢乐的一年。
'''.strip().split('\n')
corpus = [lcut(line) for line in corpus]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HONGJI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built succesfully.


## N-gram建模训练

### Unigram

In [3]:
unigram = Counter(word for words in corpus for word in words)

### Bigram

In [4]:
bigram = {w: Counter() for w in unigram.keys()}
for words in corpus:
    for i in range(len(words) - 1):
        bigram[words[i]][words[i + 1]] += 1

### Trigram

In [5]:
trigram = {words[i]+words[i+1]: Counter() for words in corpus for i in range(len(words) - 1)}
for words in corpus:
    for i in range(len(words) - 2):
        trigram[words[i]+words[i+1]][words[i+2]] += 1

## 文本生成

### Bigram文本生成

In [6]:
def generate_text_bigram(first_word, text_length=60, dof=3):
    if first_word not in unigram:
        first_word = choice(unigram.keys())
    text = first_word
    next_words = sorted(bigram[first_word], key=lambda w: bigram[first_word][w])  # 候选词
    for _ in range(text_length):  # text_length限制文本长度
        if not next_words:
            break  # 没有候选词时，退出循环
        next_word = choice(next_words[:dof])  # 选出下个词（DOF：degree of freedom）
        text += next_word
        next_words = sorted(bigram[next_word], key=lambda w: bigram[next_word][w])
    print('\033[034mBigram文本生成：\033[0m' + text)

### Trigram文本生成

In [7]:
def generate_text_trigram(first_word, text_length=60, dof=3):
    if first_word not in unigram:
        first_word = choice(list(unigram))  # 第一个词
    second_words = sorted(bigram[first_word], key=lambda w: bigram[first_word][w])  # Bigram候选词
    second_word = choice(second_words[:dof]) if second_words else ''  # 第二个词
    text = [first_word, second_word]
    next_words = sorted(trigram[first_word + second_word], key=lambda w: trigram[first_word+second_word][w])  # Trigram候选词
    for i in range(text_length):
        if not next_words:
            next_words = sorted(bigram[text[-1]], key=lambda w: bigram[text[-1]][w])  # Trigram没有候选词时，进行Bigram
            if not next_words:
                break  # Bigram没有候选词时，退出循环
        next_word_1 = choice(next_words[:dof])
        text.append(next_word_1)
        next_word_2 = ''.join(text[-2:])
        next_words = sorted(trigram[next_word_2], key=lambda w: trigram[next_word_2][w])
    print('\033[035mTrigram文本生成：\033[0m' + ''.join(text))

### 效果对比

In [8]:
def generate_text(first_word, text_length=70, dof=7):
    generate_text_bigram(first_word, text_length, dof)
    generate_text_trigram(first_word, text_length, dof)

In [9]:
generate_text('微笑')

Bigram文本生成：微笑是飞舞的点点滴滴，永享幸福平安，让无奈，操不想走的情感很重要，操的情感轨迹，永享幸福平安！
Trigram文本生成：微笑是个搬运工，可以随时搬走压力；微笑是个维修工，可以认真扫除烦忧。朋友，惟愿你天天开心，永享幸福！


In [10]:
generate_text('阳光')

Bigram文本生成：阳光洒细雨，走不想做的事，请每天保持微笑是个清洁工，爱的事，走的人生短短几十载，请每天保持微笑，操不想爱不想走不想操的人，让你作伴，做不想爱不想走的情感很重要，请每天保持微笑，让你解烦，做不想操的友情
Trigram文本生成：阳光洒在心中，惊醒一帘幽梦；问候如沐清风，为你解烦，快乐为你殷勤打扇；闲暇时，我用问候，化作无限的美好；一份思念，雨中的雷声很大，可是多不过我的心。还好有你这个朋友陪在身边，让无奈的人生平添了许多的快乐，永远美丽又


# 深度学习实现文本生成
https://blog.csdn.net/Yellow_python/article/details/86788587

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, GlobalMaxPool1D, Dense

Using TensorFlow backend.


#### 配置

In [2]:
window = 32  # 滑窗大小
filters = 25  # 卷积录波器数量
kernel_size = 5  # 卷积核大小
times = 8  # 训练总次数
batch_size = 512
epochs = 200

#### 语料

In [3]:
seq_chr = """
《长恨歌》——白居易
汉皇重色思倾国，御宇多年求不得。杨家有女初长成，养在深闺人未识。天生丽质难自弃，一朝选在君王侧。
回眸一笑百媚生，六宫粉黛无颜色。春寒赐浴华清池，温泉水滑洗凝脂。侍儿扶起娇无力，始是新承恩泽时。
云鬓花颜金步摇，芙蓉帐暖度春宵。春宵苦短日高起，从此君王不早朝。承欢侍宴无闲暇，春从春游夜专夜。
后宫佳丽三千人，三千宠爱在一身。金屋妆成娇侍夜，玉楼宴罢醉和春。姊妹弟兄皆列土，可怜光彩生门户。
遂令天下父母心，不重生男重生女。骊宫高处入青云，仙乐风飘处处闻。缓歌慢舞凝丝竹，尽日君王看不足。
渔阳鼙鼓动地来，惊破霓裳羽衣曲。九重城阙烟尘生，千乘万骑西南行。翠华摇摇行复止，西出都门百余里。
六军不发无奈何，宛转蛾眉马前死。花钿委地无人收，翠翘金雀玉搔头。君王掩面救不得，回看血泪相和流。
黄埃散漫风萧索，云栈萦纡登剑阁。峨嵋山下少人行，旌旗无光日色薄。蜀江水碧蜀山青，圣主朝朝暮暮情。
行宫见月伤心色，夜雨闻铃肠断声。天旋地转回龙驭，到此踌躇不能去。马嵬坡下泥土中，不见玉颜空死处。
君臣相顾尽沾衣，东望都门信马归。归来池苑皆依旧，太液芙蓉未央柳。芙蓉如面柳如眉，对此如何不泪垂。
春风桃李花开日，秋雨梧桐叶落时。西宫南内多秋草，落叶满阶红不扫。梨园弟子白发新，椒房阿监青娥老。
夕殿萤飞思悄然，孤灯挑尽未成眠。迟迟钟鼓初长夜，耿耿星河欲曙天。鸳鸯瓦冷霜华重，翡翠衾寒谁与共。
悠悠生死别经年，魂魄不曾来入梦。临邛道士鸿都客，能以精诚致魂魄。为感君王辗转思，遂教方士殷勤觅。
排空驭气奔如电，升天入地求之遍。上穷碧落下黄泉，两处茫茫皆不见。忽闻海上有仙山，山在虚无缥渺间。
楼阁玲珑五云起，其中绰约多仙子。中有一人字太真，雪肤花貌参差是。金阙西厢叩玉扃，转教小玉报双成。
闻道汉家天子使，九华帐里梦魂惊。揽衣推枕起徘徊，珠箔银屏迤逦开。云鬓半偏新睡觉，花冠不整下堂来。
风吹仙袂飘飖举，犹似霓裳羽衣舞。玉容寂寞泪阑干，梨花一枝春带雨。含情凝睇谢君王，一别音容两渺茫。
昭阳殿里恩爱绝，蓬莱宫中日月长。回头下望人寰处，不见长安见尘雾。惟将旧物表深情，钿合金钗寄将去。
钗留一股合一扇，钗擘黄金合分钿。但教心似金钿坚，天上人间会相见。临别殷勤重寄词，词中有誓两心知。
七月七日长生殿，夜半无人私语时。在天愿作比翼鸟，在地愿为连理枝。天长地久有时尽，此恨绵绵无绝期。
""".replace('\n', '').replace('《长恨歌》——白居易', '')

#### 数据预处理

In [4]:
len_seq = len(seq_chr)
chr_set = set(seq_chr)  # 字库
len_chr = len(chr_set)
print('语料长度', len_seq, '字汇量', len_chr)
chr2id = {c: i for i, c in enumerate(chr_set)}
id2chr = {i: c for c, i in chr2id.items()}
seq_id = [chr2id[c] for c in seq_chr]  # 文字序列 --> 索引序列

语料长度 960 字汇量 467


#### 输入层和标签

In [5]:
reshape = lambda x: np.reshape(x, (-1, window, 1)) / len_chr
x = [seq_id[i: i + window] for i in range(len_seq - window)]
x = reshape(x)
y = [seq_id[i + window] for i in range(len_seq - window)]
y = to_categorical(y, num_classes=len_chr)
print('x.shape', x.shape, 'y.shape', y.shape)

x.shape (928, 32, 1) y.shape (928, 467)


#### 建模

In [6]:
model = Sequential()
model.add(Conv1D(filters, kernel_size * 3, activation='relu'))
model.add(MaxPool1D())
model.add(Conv1D(filters * 2, kernel_size, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(len_chr, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


#### 随机采样

In [7]:
def draw_sample(predictions, temperature):
    pred = predictions.astype('float64')  # 提高精度防报错
    pred = np.log(pred) / temperature
    pred = np.exp(pred)
    pred = pred / np.sum(pred)
    pred = np.random.multinomial(1, pred, 1)
    return np.argmax(pred)

#### 预测

In [8]:
def predict(t, pred=None):
    if pred is None:
        randint = np.random.randint(len_seq - window)
        pred = seq_id[randint: randint + window]
    if t:
        print('随机采样，温度：%.1f' % t)
        sample = draw_sample
    else:
        print('贪婪采样')
        sample = np.argmax
    for _ in range(window):
        x_pred = reshape(pred[-window:])
        y_pred = model.predict(x_pred)[0]
        i = sample(y_pred, t)
        pred.append(i)
    text = ''.join([id2chr[i] for i in pred[-window:]])
    print('\033[034m%s\033[0m' % text)

#### 训练

In [9]:
for e in range(times):
    model.fit(x, y, batch_size, epochs, verbose=0)
    print(str(e + 1).center(window * 2, '-'))
    # 训练效果展示
    for t in (None, 1, 1.5, 2):
        predict(t)
print('complete'.center(window * 2, '-'))
for t in (None, 1, 1.5, 2):
    predict(t, seq_id[-window:])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
-------------------------------1--------------------------------
贪婪采样
，。。。。。，，，，，，，，，，，，。。。。子，，，，，，，，，
随机采样，温度：1.0
转梦鸟不登字谢信。王差殷颜阙霜头入旌，罢人尽摇花眸芙六百落醉，驭
随机采样，温度：1.5
堂上栈气马花驭悄闲可里重六屋龙龙虚寄缥龙光秋无怜惊芙弃凝蜀士来黄
随机采样，温度：2.0
驭电孤珑百竹仙羽埃上银下金宫睡色足汉蛾渺西阑之足躇三孤回碧裳泥遍
-------------------------------2--------------------------------
贪婪采样
搔头不头不掩面救泪得，新，花泪相相相。黄，马马，，，玉楼愿成成。
随机采样，温度：1.0
闻歌缓歌慢舞花尽死日尽监尽银相死短旧尽钿仙翠闲暇真一惊惊雪雪暮枝
随机采样，温度：1.5
。寄。一凝求睇六股军其别爱尘新时恩池多千长千皆无，夜花在私在时不
随机采样，温度：2.0
花地男君王男雨相此君王铃生生，色足爱之娥。。骊监玉伤有人，。宠在
-------------------------------3--------------------------------
贪婪采样
，不重生男重生女。骊宫高处入青云，仙乐风飘处处闻君缓歌慢舞凝丝竹
随机采样，温度：1.0
看玉泪相和流。黄埃散漫风萧索，侍栈萦纡登剑枝。峨嵋山下少。时，无
随机采样，温度：1.5
从重游不专夜。后宫色云重千在千三芙楼不翠春爱开春云垂姊经，千复万
随机采样，温度：2.0
散。秋，人生梧桐钗入归莱相宫君土九间此寰来夜寄见上死，六。天无，
-------------------------------4--------------------------------
贪婪采样
不扫。梨园弟子白发新，椒房阿监青娥老。夕殿萤飞思悄然，孤灯挑尽未
随机采样，温度：1.0
钿坚，天上天间会相。。临。人处心。风，，未。不堂不。春。地死。，
随机采样，温度：1.5
